In [ ]:
import ee
import geemap
import pandas as pd
import io
import base64
import requests
import zipfile
from tqdm import notebook
import plotly
import plotly.graph_objs as go
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from IPython.display import clear_output

In [ ]:
Map = geemap.Map(center=[20,78.5], zoom=4, add_google_map=False)
Map.add_basemap('HYBRID')
Map.add_basemap('ROADMAP')

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

start_date_picker = widgets.DatePicker(description='Select Start Date:', disabled=False, \
                                       style = style,layout=widgets.Layout(width='100%'))
end_date_picker = widgets.DatePicker(description='Select End Date: ', disabled=False, \
                                     style = style, layout=widgets.Layout(width='100%'))

no_of_largest_polygons = widgets.IntText(value=1,disabled=False, style=style, layout=widgets.Layout(width='100%'), \
                                        description="Set no of polygons for defining maximum extent mask:")

check_buffer = widgets.Checkbox(value=False,description='',disabled=False,indent=False, layout=widgets.Layout(width='6%'))

apply_buffer = widgets.IntText(description='Apply buffer to the drawn feature:', \
                                        value=1500, style=style, layout=widgets.Layout(width='94%'), disabled=True)

show_buffer = widgets.Button(description='Visualize Buffer', button_style='primary', \
                               tooltip='Tool for visualizing buffer before the main processing', \
                               layout=widgets.Layout(width='100%'), style=style, disabled=True)

run = widgets.Button(description='Start Processing', button_style='primary', \
                     tooltip='Calculates instantaneous surface water extents for the defined time-period and AOI', \
                     layout=widgets.Layout(width='100%'), style=style)

gen_links_label = widgets.Label(value="Generate download links of the processed final outputs :")

max_extent_file = widgets.Button(description='Maximum Extent Water Mask Shapefile', button_style='primary', \
                         tooltip='Generates download link for exporting maximum extent water mask shapefile (.zip)', \
                         layout=widgets.Layout(width='100%'), style=style, disabled=True)

export_csv = widgets.Button(description='Time-Series of Water Surface Area', button_style='primary', \
                         tooltip='Generate download links for exporting time-series of water surface area (.csv)', \
                         layout=widgets.Layout(width='100%'), style=style, disabled=True)

export_shp = widgets.Button(description='Time-Series of Surface Water Extent Shapefiles', button_style='primary', \
                         tooltip='Generate download links for exporting time-series of surface water extent shapefiles (.zip)', \
                         layout=widgets.Layout(width='100%'), style=style, disabled=True)

full_widget = widgets.VBox([start_date_picker, end_date_picker, \
                           widgets.HBox([check_buffer,apply_buffer], \
                                         layout=widgets.Layout(justify_content="flex-start",width='100%')), \
                           show_buffer, no_of_largest_polygons, run, gen_links_label, \
                           max_extent_file, export_csv, export_shp], \
                           layout=widgets.Layout(justify_content="space-around"))

wsa_df = pd.DataFrame()

def getInfo_fn(var):
    try:
        new_var = var.getInfo()
    except Exception:
        print('Error: Earth Engine quota exceeded.....Retrying')
        try:
            new_var = var.getInfo()
        except Exception:
            print('Error: Earth Engine quota exceeded. Try processing again or reducing the AOI or time-period of analysis.')
            pass
    return new_var

def activate_buffer(b):
    if apply_buffer.disabled == True:
        apply_buffer.disabled = False
        show_buffer.disabled = False
    else:
        apply_buffer.disabled = True
        show_buffer.disabled = True
    def buffer_clicked(b):
        with output_widget:
            output_widget.clear_output()
            print('Buffering')
            if Map.user_roi:
                aoi = getInfo_fn(Map.user_roi.type())
                if aoi == 'Point':
                    print('Error: Point feature cannot be used to define AOI')
                else:
                    roi = Map.user_roi.buffer(apply_buffer.value)
                    Map.addLayer(roi,{},'AOI')
                    print('Done')
    show_buffer.on_click(buffer_clicked)
check_buffer.observe(activate_buffer)

def area(shp):
    shp = ee.Feature(shp)
    area = shp.area(1)
    return shp.set('Area',area)

def run_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Processing Started...')
        Map.default_style = {'cursor': 'wait'}
        
        try:
            if Map.user_roi:
                aoi = getInfo_fn(Map.user_roi.type())
                if aoi == 'Polygon':
                    if check_buffer.value:
                        roi = Map.user_roi.buffer(apply_buffer.value)
                    else:
                        roi = Map.user_roi
                elif aoi == 'LineString':
                    if check_buffer.value:
                        roi = Map.user_roi.buffer(apply_buffer.value)
                    else:
                        output_widget.clear_output()
                        print('Error: LineString must be buffered to define AOI')
                        Map.default_style = {'cursor': 'default'}
                        return
                elif aoi == 'Point':
                    output_widget.clear_output()
                    print('Error: Point feature cannot be used to define AOI')
                    Map.default_style = {'cursor': 'default'}
                    return
            else:
                output_widget.clear_output() 
                print('Error: AOI not detected')
                Map.default_style = {'cursor': 'default'}
                return
            
            print('Checking availability of Sentinel-1 data for the specified time period and AOI')
            start_date = start_date_picker.value.strftime('%Y-%m-%d')
            end_date = end_date_picker.value.strftime('%Y-%m-%d')
            img_coll = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(start_date, end_date).filterBounds(roi) \
                   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            srtm = ee.Image("USGS/SRTMGL1_003") 
            
            start = ee.Date(start_date)
            end = ee.Date(end_date)
            diff = end.difference(start, 'day')
            date_range = ee.List.sequence(0, diff.subtract(1)).map(lambda day: start.advance(day,'day'))
            
            dates = img_coll.aggregate_array('system:time_start')
            dates_file = dates.map(lambda d: ee.Date(d).format('yyyy_MM_dd'))
            dates_file = list(dict.fromkeys(getInfo_fn(dates_file)))
            
            img_clip_coll = img_coll.map(lambda img: img.clip(roi))
            
            num = getInfo_fn(img_clip_coll.size())

            final_img_coll = img_clip_coll
            
            tot_no = getInfo_fn(final_img_coll.length())
            if tot_no == 0:
                print('Error: Defined AOI is too large to be covered by Sentinel-1 overpasses.'\
                      '\nTry dividing your AOI into sub-sections or sub-reaches and process separately.')
                Map.default_style = {'cursor': 'default'}
                return
            else:
                print('No. of images identified: '+str(tot_no))
            
            print('Processing images')
            def filterSpeckles(img):
                vv = img.select('VV') 
                vv_filtered = vv.focal_median(100,'circle','meters').rename('VV_Filtered') 
                return img.addBands(vv_filtered) 
            final_coll = final_img_coll.map(filterSpeckles)
            
            def otsu(histogram):
                no = ee.Array(ee.Dictionary(histogram).get('histogram'))
                means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
                size = means.length().get([0])
                total = no.reduce(ee.Reducer.sum(), [0]).get([0])
                red_sum = means.multiply(no).reduce(ee.Reducer.sum(), [0]).get([0])
                avg = red_sum.divide(total)
                ind = ee.List.sequence(1, size)

                def bss(i):
                    a_nos = no.slice(0, 0, i)
                    a_no = a_nos.reduce(ee.Reducer.sum(), [0]).get([0])
                    a_means = means.slice(0, 0, i)
                    a_mean = a_means.multiply(a_nos).reduce(ee.Reducer.sum(), [0]).get([0]).divide(a_no)
                    b_no = total.subtract(a_no)
                    b_mean = red_sum.subtract(a_no.multiply(a_mean)).divide(b_no)
                    return (a_no.multiply(a_mean.subtract(avg).pow(2)).add(b_no.multiply(b_mean.subtract(avg).pow(2)))).divide(total)
                bss = ind.map(bss)

                return means.sort(bss).get([-1])
          
            n = getInfo_fn(final_coll.size())
            img_list = final_coll.toList(n)
            WM_final = ee.List([])
            for i in range(n):
                img = ee.Image(img_list.get(i))
                hist = img.select('VV_Filtered').reduceRegion(**{'reducer': ee.Reducer.histogram(255, 2), \
                                                                'geometry': roi, 'scale': 20, 'bestEffort': True});
                threshold = otsu(hist.get('VV_Filtered'))
                WM = img.select('VV_Filtered').lt(threshold)
                WM_final = WM_final.add(WM)
            
            print('Generating maximum extent water mask')
            WM_collection = ee.ImageCollection.fromImages(WM_final)
            max_extent_mask = WM_collection.qualityMosaic('VV_Filtered').select('VV_Filtered')
            Map.addLayer(max_extent_mask, {'min': 0, 'max': 1, 'palette': ['#FFFFFF','#0000FF']}, \
                         'Maximum Extent Water Mask')
            
            shapefile = WM_collection.qualityMosaic('VV_Filtered').select('VV_Filtered') \
                                     .reduceToVectors(**{'geometry': roi, 'scale': 20,'maxPixels': 1000000000, \
                                                         'bestEffort': True, 'eightConnected': False})

            max_extent_shp = shapefile.map(area).sort('Area',False)
            n1 = max_extent_shp.size()
            max_extent_poly_list = max_extent_shp.toList(n1)
            no_of_poly = no_of_largest_polygons.value
            max_extent_filtered = ee.FeatureCollection(max_extent_poly_list.slice(0,no_of_poly))
            Map.addLayer(max_extent_filtered, {'color': 'FF0000', 'maxPixels': 100000000}, \
                         'Maximum Extent Water Mask Shapefile')
            
            def RW_generate(img):
                shp = img.select('VV_Filtered') \
                         .reduceToVectors(**{'geometry': max_extent_filtered.geometry(), \
                         'scale': 20, 'maxPixels': 1000000000, 'bestEffort': True, 'eightConnected': False})
                return shp
            final_RW = WM_collection.map(RW_generate)
            shp_list = final_RW.toList(n)
            
            wsa = ee.List([])
            for i in range(n):
                area_poly = ee.FeatureCollection(shp_list.get(i)).map(area)
                area_shp = area_poly.aggregate_sum('Area')
                area_shp = area_shp.multiply(ee.Number(10).pow(ee.Number(-6)))
                wsa = wsa.add(area_shp)
            print('Generating time-series data')
            wsa_new = getInfo_fn(wsa)
            global wsa_df
            wsa_df = pd.DataFrame({'Dates':filtered_dates, 'WSA in square kilometers':wsa_new})
            
            print('Processing Completed!!!')
            print("Check 'View Time-Series Plot' tab to visualize time-series plot.")
            Map.default_style = {'cursor': 'default'}
                    
        except Exception as e:
            print(e)
            print('An error occurred during computation.\nTry processing again or reducing the AOI or time-period of analysis.')
        
        Map.default_style = {'cursor': 'default'}

run.on_click(run_clicked)

In [ ]:
full_widget

In [ ]:
Map

In [ ]:
view_text = widgets.HTML(value="Click here to visualize the time-series plot of Water Surface Area .")
view_plot = widgets.Button(description='View Time-Series Plot', button_style='primary', \
                     tooltip='View time-series plot of water surface area', \
                     layout=widgets.Layout(width='15%'), style=style)
out = widgets.Output()
def view_plot_clicked(b):
    if not wsa_df.empty:
        data = [go.Scatter(x = wsa_df['Dates'], y = wsa_df['WSA in square kilometers'])]
        layout = go.Layout(title = {'text': '<b>Water Surface Area (WSA) - Time Series</b>','y':0.9,'x':0.5, \
                                    'xanchor': 'center','yanchor': 'top'}, xaxis_title = '<b>Dates</b>', \
                           yaxis_title = '<b>WSA in square kilometers</b>', xaxis = {'dtick': 4})
        with out:
            out.clear_output()
            plotly.offline.iplot({"data":data, "layout":layout})
    else:
        with out:
            out.clear_output()
            print("Error: Complete the processing to visualize plot.")
view_plot.on_click(view_plot_clicked)
widgets.HBox([view_text,view_plot],layout=widgets.Layout(justify_content="flex-start"))

In [ ]:
out